# AI Learning 07 - 4. Cross Validation

### K-fold Cross Validation

In [139]:
X.shape, y.shape

((1460, 30), (1460,))

In [ ]:
# 지난번에 training set과 test set을 나눈 코드
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [142]:
# 위에서 test set을 10%로 나눠놨다.
# 이제 10%로 나눈 test set을 10번에 걸쳐서 서로 다른 test set으로 만들기로 함.
K = 10
data_size = int(len(X) / K)

In [143]:
data_size

146

In [144]:
# 자주 쓰이는 개념이라 사이킷런에 KFold로 만들어져 있음.
from sklearn.model_selection import KFold

In [145]:
# 앞으로 kf를 사용해서 split을 하면서 학습을 시킬 것이다.
kf = KFold(n_splits=K)

In [146]:
# data preprocessing
# split한 뒤에 전처리하는 것은 번거로우니 먼저 X, y를 통째로 전처리 해준다.
# 지난번처럼 간단히 column을 0에서 1 사이의 값으로 normalization을 해준다.

x_min_max_scaler = MinMaxScaler()
x_min_max_scaler.fit(X)
scaled_X = x_min_max_scaler.transform(X)

y_min_max_scaler = MinMaxScaler()
y_min_max_scaler.fit(np.array(y).reshape(-1, 1))
scaled_y = y_min_max_scaler.transform(np.array(y).reshape(-1, 1))

In [147]:
scaled_X.shape, scaled_y.shape

((1460, 30), (1460, 1))

In [163]:
# kf.split()에 scaled_X를 넣어주면 자동으로 index를 뽑아준다.
# 최종적인 지표는 rmse의 값들을 평균내줘야한다.
rmses = []

for train_index, test_index in kf.split(scaled_X):
  # print("TRAIN:", train_index, "TEST:", test_index)

  X_train, X_test = scaled_X[train_index], scaled_X[test_index]
  y_train, y_test = scaled_y[train_index], scaled_y[test_index]

  # print(X_train.shape, X_test.shape)
  # print(y_train.shape, y_test.shape)

  # training
  model = Lasso()
  model = model.fit(X_train, y_train)

  # evaluation
  pred = model.predict(X_test).reshape((-1, 1))
  pred = y_min_max_scaler.inverse_transform(pred)
  rmse = np.sqrt(metrics.mean_squared_error(y_test, pred))

  print(rmse)
  print("---------------------")

  rmses.append(rmse)

181632.6112898907
---------------------
180027.94663797648
---------------------
180696.21069777504
---------------------
180689.21822073552
---------------------
180332.03110818347
---------------------
180697.65972653756
---------------------
181755.2208892524
---------------------
181831.19976882864
---------------------
180042.74895388487
---------------------
181505.08382085158
---------------------


In [164]:
rmses

[181632.6112898907,
 180027.94663797648,
 180696.21069777504,
 180689.21822073552,
 180332.03110818347,
 180697.65972653756,
 181755.2208892524,
 181831.19976882864,
 180042.74895388487,
 181505.08382085158]

In [165]:
# rmses에 넣어둔 값들을 평균냄.
np.mean(rmses)

180920.99311139164

이런 방법이 data set이 적을 때 일반적으로 사용하는 평가 방법이다.<br>
k개로 잘라서 각각 평가를 한 뒤에 평가 지표들을 평균을 낸다.